# Importing the Data

In [19]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', 100)

In [3]:
# load the csvs - it is available open source and online
iaps = pd.read_csv('iaps.csv')
sessions = pd.read_csv('sessions.csv')
spendevents = pd.read_csv('spendevents.csv')
users = pd.read_csv('users.csv')

In [4]:
print(len(iaps))
print(len(sessions))
print(len(spendevents))
print(len(users))

6685
722955
107764
22576


# Exploratory Analysis

## IAPS Analysis

In [15]:
# display data
iaps

,user_id,ts,date,prod_name,prod_type,rev
0,7480,2019-03-04 08:15:49,2019-03-04,iap_1_gems_2,gems,760
1,7480,2019-03-04 08:24:15,2019-03-04,iap_1_passes_2,chapterPasses,760
2,7480,2019-03-04 22:49:08,2019-03-04,iap_1_gems_1,gems,410
3,2466,2019-03-06 00:16:48,2019-03-06,iap_1_gems_2,gems,760
4,22001,2019-03-06 09:13:45,2019-03-06,iap_1_gems_2,gems,760
...,...,...,...,...,...,...
6680,18174,2019-04-21 18:50:42,2019-04-21,iap_1_passes_10,chapterPasses,3555
6681,18174,2019-04-21 18:51:29,2019-04-21,iap_1_passes_1,chapterPasses,410
6682,18174,2019-04-21 20:08:06,2019-04-21,iap_1_passes_10,chapterPasses,3555
6683,18174,2019-04-22 17:07:19,2019-04-22,iap_1_passes_2,chapterPasses,760


In [14]:
iaps.dtypes

user_id       int64
ts           object
date         object
prod_name    object
prod_type    object
rev           int64
dtype: object

#### Analysis of how much each product costs

In [26]:
# These are all the packages the users can buy
iaps['prod_name'].value_counts()

iap_1_gems_2          2599
iap_1_passes_2        1064
iap_1_gems_5           976
iap_1_gems_1           693
iap_1_passes_1         576
iap_1_gems_10          231
iap_1_passes_5         226
iap_1_passes_10        163
iap_1_gems_20           69
iap_value_pack_004      46
iap_value_pack_008      13
iap_value_pack_001      12
iap_value_pack_005       5
iap_1_gems_50            3
iap_value_pack_010       3
iap_1_gems_100           2
iap_value_pack_003       2
iap_value_pack_009       1
iap_value_pack_007       1
Name: prod_name, dtype: int64

In [7]:
# These are the amount of gems that the user bought. These should correspond to the prod_name packages
iaps['rev'].value_counts()

760      3663
410      1281
1810     1207
3560      234
3555      163
7060       69
1460       46
2860       13
17560       3
35060       2
1110        2
2510        1
3210        1
Name: rev, dtype: int64

In [10]:
iaps['rev'][2]

410

In [13]:
prod_list = {}

for i, val in enumerate(iaps['prod_name']):
    if val not in prod_list:
        prod_list[val] = iaps['rev'][i]
        # print(val, iaps['rev'][i])

prod_list

{'iap_1_gems_2': 760,
 'iap_1_passes_2': 760,
 'iap_1_gems_1': 410,
 'iap_1_passes_1': 410,
 'iap_1_gems_5': 1810,
 'iap_1_passes_5': 1810,
 'iap_1_gems_10': 3560,
 'iap_1_passes_10': 3555,
 'iap_1_gems_20': 7060,
 'iap_value_pack_004': 1460,
 'iap_value_pack_008': 2860,
 'iap_1_gems_50': 17560,
 'iap_value_pack_001': 410,
 'iap_value_pack_005': 1810,
 'iap_value_pack_003': 1110,
 'iap_value_pack_010': 3560,
 'iap_1_gems_100': 35060,
 'iap_value_pack_007': 2510,
 'iap_value_pack_009': 3210}

#### Date Analysis

In [16]:
# Purchase data seems to be from March to May in 2019
iaps.sort_values(by=['ts'])

,user_id,ts,date,prod_name,prod_type,rev
3460,19248,2019-03-01 00:24:52,2019-03-01,iap_1_gems_2,gems,760
3461,19248,2019-03-01 00:36:14,2019-03-01,iap_1_gems_2,gems,760
3442,12752,2019-03-01 01:18:04,2019-03-01,iap_1_passes_1,chapterPasses,410
2074,10387,2019-03-01 01:19:41,2019-03-01,iap_1_gems_2,gems,760
5431,13356,2019-03-01 03:35:04,2019-03-01,iap_1_passes_2,chapterPasses,760
...,...,...,...,...,...,...
2963,20401,2019-05-05 17:26:24,2019-05-05,iap_1_passes_2,chapterPasses,760
3604,15446,2019-05-05 18:04:08,2019-05-05,iap_value_pack_010,valuePack,3560
5430,15443,2019-05-05 20:40:23,2019-05-05,iap_1_passes_10,chapterPasses,3555
3605,15446,2019-05-05 22:12:51,2019-05-05,iap_1_gems_2,gems,760


In [20]:
# There seems to be less data as time goes on
iaps['date'].value_counts()

2019-03-06    873
2019-03-07    766
2019-03-05    653
2019-03-08    451
2019-03-04    433
2019-03-03    356
2019-03-02    333
2019-03-09    260
2019-03-01    188
2019-03-10    173
2019-03-11    155
2019-03-12    131
2019-03-13    131
2019-03-19    113
2019-03-20     91
2019-03-14     83
2019-03-18     64
2019-03-27     63
2019-03-16     59
2019-04-02     56
2019-03-15     52
2019-04-03     50
2019-03-29     49
2019-03-26     49
2019-03-25     48
2019-03-21     44
2019-04-09     44
2019-03-28     43
2019-03-17     43
2019-03-31     42
2019-03-24     42
2019-04-16     42
2019-03-22     39
2019-04-04     38
2019-04-01     37
2019-04-22     34
2019-04-17     32
2019-04-06     30
2019-04-21     30
2019-04-14     30
2019-03-23     30
2019-04-05     29
2019-04-13     27
2019-04-12     26
2019-04-15     26
2019-03-30     23
2019-04-08     23
2019-04-10     21
2019-04-11     21
2019-04-07     21
2019-04-30     20
2019-04-23     19
2019-04-27     19
2019-04-24     17
2019-04-20     17
2019-04-28

## Spendevents Exploratory Analysis

In [32]:
# display data
spendevents

,user_id,ts,date,story,chapter,spendtype,currency,amount
0,9829,2019-03-01 03:03:04,2019-03-01,story_1,0,earnGemsCounter,gems,-22
1,13757,2019-03-01 03:35:53,2019-03-01,story_1,0,earnGemsCounter,gems,-22
2,13757,2019-03-01 03:52:10,2019-03-01,story_2,0,earnGemsCounter,gems,-22
3,10009,2019-03-01 04:10:00,2019-03-01,story_1,0,earnGemsCounter,gems,-22
4,10009,2019-03-01 04:26:46,2019-03-01,story_2,0,earnGemsCounter,gems,-22
...,...,...,...,...,...,...,...,...
107759,15156,2019-04-23 02:31:46,2019-04-23,story_3,0,earnGemsCounter,gems,0
107760,15156,2019-04-23 02:31:59,2019-04-23,story_3,0,earnGemsCounter,gems,0
107761,15156,2019-04-23 02:32:04,2019-04-23,story_3,0,earnGemsCounter,gems,0
107762,15156,2019-04-23 02:32:11,2019-04-23,story_3,0,earnGemsCounter,gems,0


In [37]:
# filter to user id == 2062 due to assignment instructions
filtered_spendevents = spendevents['user_id'] == 2062

# order by ts so we can see all three spendtypes (according to assignment instructions)
spendevents[filtered_spendevents].sort_values(by=['ts'])

,user_id,ts,date,story,chapter,spendtype,currency,amount
15694,2062,2019-03-02 20:01:22,2019-03-03,story_1,0,earnGemsCounter,gems,-22
15721,2062,2019-03-03 03:50:45,2019-03-03,story_1,5,IAP,gems,-73
15722,2062,2019-03-03 03:50:58,2019-03-03,story_1,5,IAP,gems,-73
15723,2062,2019-03-03 03:51:00,2019-03-03,story_1,5,premiumChoice,gems,64
15724,2062,2019-03-03 04:07:34,2019-03-03,story_1,6,IAP,gems,-73
15725,2062,2019-03-03 04:07:37,2019-03-03,story_1,6,premiumChoice,gems,55
15726,2062,2019-03-03 04:24:58,2019-03-03,story_1,8,premiumChoice,gems,27


In [39]:
# the only currency is gems. This column is irrelevant
spendevents['currency'].value_counts()

gems    107764
Name: currency, dtype: int64

In [23]:
# shows that there is a 4th option for spendtype that wasn't discussed in the assignment (valuepack)
spendevents['spendtype'].value_counts()

earnGemsCounter    87235
premiumChoice      14231
IAP                 4915
valuepack             39
Name: spendtype, dtype: int64

In [41]:
# filter to spendtype == valuepack
filtered_spendtype = spendevents['spendtype'] == 'valuepack'

# we can see that value pack is another pack where the user spends money to buy gems
spendevents[filtered_spendtype].head()

,user_id,ts,date,story,chapter,spendtype,currency,amount
27107,9984,2019-04-15 20:16:49,2019-04-16,story_15,10,valuepack,gems,-322
32772,5782,2019-04-17 05:33:26,2019-04-17,story_1,4,valuepack,gems,-82
52405,5987,2019-04-17 05:02:48,2019-04-17,story_4,24,valuepack,gems,-217
52406,5987,2019-04-17 05:02:48,2019-04-17,story_4,24,valuepack,gems,-217
57519,8874,2019-04-17 07:46:03,2019-04-17,story_2,0,valuepack,gems,-217


## Sessions Exploratory Analysis

Preliminary analysis should be to find the average time it takes a user (that eventually pays for gems)
We can look at the ts for sessions and spendevents dataframe to figure this out 

In [44]:
# shows some users played a lot of sessions
sessions.sort_values(by=["session_num"])

,user_id,ts,date,session_num,last_session_termination_type
0,14067,2019-03-01 00:06:50,2019-03-01,1,NaN
325489,5532,2019-03-05 11:51:31,2019-03-05,1,NaN
325497,2505,2019-03-05 12:41:40,2019-03-05,1,NaN
325510,18717,2019-03-05 13:57:08,2019-03-05,1,NaN
325511,10274,2019-03-05 14:06:29,2019-03-05,1,NaN
...,...,...,...,...,...
721443,599,2019-05-05 21:45:33,2019-05-05,1915,NaN
721444,599,2019-05-05 21:46:03,2019-05-05,1916,NaN
721445,599,2019-05-05 21:48:38,2019-05-05,1917,NaN
721446,599,2019-05-05 21:50:00,2019-05-05,1918,NaN


In [46]:
# over the course of 2 months, user 599 played 1919 times
filtered_user = sessions['user_id'] == 599
sessions[filtered_user].sort_values(by=['ts'])

,user_id,ts,date,session_num,last_session_termination_type
231455,599,2019-03-06 07:16:25,2019-03-06,1,NaN
231474,599,2019-03-06 07:39:11,2019-03-06,2,NaN
231495,599,2019-03-06 08:24:10,2019-03-06,3,NaN
231978,599,2019-03-06 08:59:19,2019-03-06,4,NaN
231993,599,2019-03-06 09:09:02,2019-03-06,5,NaN
...,...,...,...,...,...
721443,599,2019-05-05 21:45:33,2019-05-05,1915,NaN
721444,599,2019-05-05 21:46:03,2019-05-05,1916,NaN
721445,599,2019-05-05 21:48:38,2019-05-05,1917,NaN
721446,599,2019-05-05 21:50:00,2019-05-05,1918,NaN


In [63]:
# this is a useless column
sessions['last_session_termination_type'].value_counts()

Series([], Name: last_session_termination_type, dtype: int64)

## Users Exploratory Analysis

In [48]:
# seems the data only contains users who installed the game in the first week of march
users.sort_values(by=['install_date'])

,user_id,install_date,lang,country,hw_ver,os_ver
0,0,2019-03-01,en,US,"iPhone4,1",9.1
17400,17400,2019-03-01,de,DE,"iPhone6,2",9.2.1
2259,2259,2019-03-01,en,US,"iPhone8,1",9.2
11137,11137,2019-03-01,id,ID,"iPhone5,2",8.1.3
17393,17393,2019-03-01,ja,JP,"iPhone7,2",9.2.1
...,...,...,...,...,...,...
20615,20615,2019-03-07,en,US,"iPhone7,2",9.2.1
4366,4366,2019-03-07,en,US,"iPhone5,2",8.4
5359,5359,2019-03-07,es,MX,"iPod7,1",9.2
3250,3250,2019-03-07,en,US,"iPad2,2",9.2.1


In [57]:
# 55 different languages spoken by users
print('# of languages spoken by users: ', len(users["lang"].value_counts()))

# 177 different countries where our users lived
print('# of user countries: ', len(users["country"].value_counts()))

# 43 hardwear versions
print('# of hardware versions: ', len(users["hw_ver"].value_counts()))

# 31 os versions
print('# of OS versions: ', len(users["os_ver"].value_counts()))

# of languages spoken by users:  55
# of user countries:  177
# of hardware versions:  43
# of OS versions:  31


In [60]:
users["hw_ver"].value_counts()

iPhone7,2    3817
iPhone8,1    2854
iPhone6,1    2207
iPhone6,2    1483
iPhone5,2    1101
iPhone5,3    1010
iPad2,5      1004
iPhone7,1     986
iPhone8,2     904
iPhone4,1     844
iPad4,4       710
iPad4,1       589
iPod5,1       491
iPhone5,1     490
iPhone5,4     477
iPhone3,1     416
iPad5,3       346
iPad3,4       339
iPod7,1       324
iPad2,1       254
iPad2,4       247
iPhone3,3     176
iPad3,1       166
iPad5,4       154
iPad4,2       147
iPad2,2       145
iPad4,5       131
iPad4,7        98
iPad3,3        97
iPad5,1        91
iPad2,7        90
iPhone3,2      86
iPad3,6        85
iPad2,6        43
iPod4,1        33
iPad4,8        29
iPad5,2        29
iPad3,5        22
iPad6,7        21
iPad2,3        16
iPad6,8        15
iPad3,2         8
iPhone2,1       1
Name: hw_ver, dtype: int64

some features that we can use to help determine the target audience could be language, country, hw_ver
os_ver seems to overlap with hw_ver. I prefer using hw_ver due to it be more indicative of wealth

# Target Audience Analysis

## Merge on Sessions and Spendevents

Simple analysis to find the average time before their first session and the time they make their first purchase

#### Merging and Wrangling

In [71]:
# merge users and sessions so we can track of session numbers
sess_spend = pd.merge(sessions, spendevents, on='user_id', how='inner')
sess_spend

,user_id,ts_x,date_x,session_num,last_session_termination_type,ts_y,date_y,story,chapter,spendtype,currency,amount
0,14067,2019-03-01 00:06:50,2019-03-01,1,NaN,2019-03-01 00:11:17,2019-03-01,story_1,0,earnGemsCounter,gems,-22
1,14067,2019-03-01 00:06:50,2019-03-01,1,NaN,2019-03-01 10:38:04,2019-03-01,story_2,0,earnGemsCounter,gems,-22
2,14067,2019-03-01 00:22:27,2019-03-01,2,NaN,2019-03-01 00:11:17,2019-03-01,story_1,0,earnGemsCounter,gems,-22
3,14067,2019-03-01 00:22:27,2019-03-01,2,NaN,2019-03-01 10:38:04,2019-03-01,story_2,0,earnGemsCounter,gems,-22
4,14067,2019-03-01 09:42:32,2019-03-01,3,NaN,2019-03-01 00:11:17,2019-03-01,story_1,0,earnGemsCounter,gems,-22
...,...,...,...,...,...,...,...,...,...,...,...,...
11524835,9349,2019-03-08 03:46:41,2019-03-08,15,NaN,2019-03-08 01:49:33,2019-03-08,story_2,0,earnGemsCounter,gems,-22
11524836,9349,2019-03-08 03:53:20,2019-03-08,16,NaN,2019-03-08 00:02:31,2019-03-08,story_1,0,earnGemsCounter,gems,-22
11524837,9349,2019-03-08 03:53:20,2019-03-08,16,NaN,2019-03-08 01:49:33,2019-03-08,story_2,0,earnGemsCounter,gems,-22
11524838,9349,2019-03-08 03:59:26,2019-03-08,17,NaN,2019-03-08 00:02:31,2019-03-08,story_1,0,earnGemsCounter,gems,-22


In [ ]:
## exploratory analysis to see if column should be dropped
# sess_spend['story'].value_counts()

In [78]:
drop_cols = ['currency', 'chapter', 'date_y', 'date_x', 'story', 'last_session_termination_type']
sess_spend.drop(drop_cols, axis=1, inplace=True)
sess_spend


,user_id,ts_x,session_num,ts_y,spendtype,amount
0,14067,2019-03-01 00:06:50,1,2019-03-01 00:11:17,earnGemsCounter,-22
1,14067,2019-03-01 00:06:50,1,2019-03-01 10:38:04,earnGemsCounter,-22
2,14067,2019-03-01 00:22:27,2,2019-03-01 00:11:17,earnGemsCounter,-22
3,14067,2019-03-01 00:22:27,2,2019-03-01 10:38:04,earnGemsCounter,-22
4,14067,2019-03-01 09:42:32,3,2019-03-01 00:11:17,earnGemsCounter,-22
...,...,...,...,...,...,...
11524835,9349,2019-03-08 03:46:41,15,2019-03-08 01:49:33,earnGemsCounter,-22
11524836,9349,2019-03-08 03:53:20,16,2019-03-08 00:02:31,earnGemsCounter,-22
11524837,9349,2019-03-08 03:53:20,16,2019-03-08 01:49:33,earnGemsCounter,-22
11524838,9349,2019-03-08 03:59:26,17,2019-03-08 00:02:31,earnGemsCounter,-22


In [86]:
# filter on spendtypes that give revenue to Pocket Gems
IAP_value_filter = (sess_spend['spendtype'] == 'IAP') | (sess_spend['spendtype'] == 'valuepack')
sess_spend_rev = sess_spend[IAP_value_filter]
sess_spend_rev

,user_id,ts_x,session_num,ts_y,spendtype,amount
157,19284,2019-03-01 06:48:49,1,2019-03-03 10:53:20,IAP,-73
160,19284,2019-03-01 07:07:43,2,2019-03-03 10:53:20,IAP,-73
163,19284,2019-03-01 07:12:22,3,2019-03-03 10:53:20,IAP,-73
166,19284,2019-03-01 10:37:06,4,2019-03-03 10:53:20,IAP,-73
169,19284,2019-03-03 10:38:55,5,2019-03-03 10:53:20,IAP,-73
...,...,...,...,...,...,...
11524780,12243,2019-03-22 01:24:56,31,2019-03-13 16:46:19,IAP,-73
11524784,12243,2019-03-22 23:07:30,32,2019-03-13 16:46:19,IAP,-73
11524788,12243,2019-03-24 04:17:32,33,2019-03-13 16:46:19,IAP,-73
11524792,12243,2019-03-24 04:18:02,34,2019-03-13 16:46:19,IAP,-73


In [97]:
# look at dtypes. make sure ts' are datetimes
sess_spend_rev.dtypes


user_id                 int64
ts_x           datetime64[ns]
session_num             int64
ts_y           datetime64[ns]
spendtype              object
amount                  int64
dtype: object

In [95]:
# convert ts to dt
sess_spend_rev['ts_x'] = pd.to_datetime(sess_spend_rev['ts_x'])
sess_spend_rev['ts_y'] = pd.to_datetime(sess_spend_rev['ts_y'])

C:\Users\Eric\anaconda3\envs\unit2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Eric\anaconda3\envs\unit2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [107]:
# reset index for iteration
sess_spend_rev.reset_index(drop=True, inplace=True)
sess_spend_rev

,user_id,ts_x,session_num,ts_y,spendtype,amount
0,19284,2019-03-01 06:48:49,1,2019-03-03 10:53:20,IAP,-73
1,19284,2019-03-01 07:07:43,2,2019-03-03 10:53:20,IAP,-73
2,19284,2019-03-01 07:12:22,3,2019-03-03 10:53:20,IAP,-73
3,19284,2019-03-01 10:37:06,4,2019-03-03 10:53:20,IAP,-73
4,19284,2019-03-03 10:38:55,5,2019-03-03 10:53:20,IAP,-73
...,...,...,...,...,...,...
669495,12243,2019-03-22 01:24:56,31,2019-03-13 16:46:19,IAP,-73
669496,12243,2019-03-22 23:07:30,32,2019-03-13 16:46:19,IAP,-73
669497,12243,2019-03-24 04:17:32,33,2019-03-13 16:46:19,IAP,-73
669498,12243,2019-03-24 04:18:02,34,2019-03-13 16:46:19,IAP,-73


#### Find Average time between first session and purchase

In [105]:
sess_spend_rev['ts_x'][157]

Timestamp('2019-03-01 06:48:49')

In [ ]:
# Create an empty list to hold all average times
purchase_times = []

# everytime a session == 1, find the average time between the 1st session and the 1st purchase time
for i, session in sess_spend_rev['session_num']:
    if session == 1:
        purc_time = sess_spend_rev['ts_x']


## Merge on Users, Sessions, Spendevents

Merge on Users, Sessions, and Spendevents since we can see a lot of features and see how that affects the target audience
IAP and Spendevents is too similar while Spendevents has a lot more data so I chose to use Spendevents


In [61]:
# merge users and sessions so we can track of session numbers
users_sess = pd.merge(users, sessions, on='user_id', how='outer')
users_sess

,user_id,install_date,lang,country,hw_ver,os_ver,ts,date,session_num,last_session_termination_type
0,0,2019-03-01,en,US,"iPhone4,1",9.1,2019-03-01 12:39:19,2019-03-01,1.0,NaN
1,0,2019-03-01,en,US,"iPhone4,1",9.1,2019-03-01 12:45:47,2019-03-01,2.0,NaN
2,0,2019-03-01,en,US,"iPhone4,1",9.1,2019-03-01 17:00:26,2019-03-01,3.0,NaN
3,0,2019-03-01,en,US,"iPhone4,1",9.1,2019-03-01 17:01:16,2019-03-01,4.0,NaN
4,0,2019-03-01,en,US,"iPhone4,1",9.1,2019-03-01 17:04:46,2019-03-01,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...
722982,22573,2019-03-05,en,GB,"iPad2,5",8.3,2019-04-07 00:07:09,2019-04-07,16.0,NaN
722983,22574,2019-03-02,en,US,"iPhone8,1",9.1,2019-03-02 18:22:19,2019-03-02,1.0,NaN
722984,22574,2019-03-02,en,US,"iPhone8,1",9.1,2019-03-02 18:22:38,2019-03-02,2.0,NaN
722985,22575,2019-03-03,en,US,"iPhone7,2",9.2.1,2019-03-03 19:14:35,2019-03-03,1.0,NaN


In [64]:
# merge users sess and spendevents to get a lot of features from each df
users_sess_spend = pd.merge(users_sess, spendevents, on='user_id', how='outer')
users_sess_spend

,user_id,install_date,lang,country,hw_ver,os_ver,ts_x,date_x,session_num,last_session_termination_type,ts_y,date_y,story,chapter,spendtype,currency,amount
0,0,2019-03-01,en,US,"iPhone4,1",9.1,2019-03-01 12:39:19,2019-03-01,1.0,NaN,2019-03-01 19:19:52,2019-03-01,story_1,0.0,earnGemsCounter,gems,-22.0
1,0,2019-03-01,en,US,"iPhone4,1",9.1,2019-03-01 12:45:47,2019-03-01,2.0,NaN,2019-03-01 19:19:52,2019-03-01,story_1,0.0,earnGemsCounter,gems,-22.0
2,0,2019-03-01,en,US,"iPhone4,1",9.1,2019-03-01 17:00:26,2019-03-01,3.0,NaN,2019-03-01 19:19:52,2019-03-01,story_1,0.0,earnGemsCounter,gems,-22.0
3,0,2019-03-01,en,US,"iPhone4,1",9.1,2019-03-01 17:01:16,2019-03-01,4.0,NaN,2019-03-01 19:19:52,2019-03-01,story_1,0.0,earnGemsCounter,gems,-22.0
4,0,2019-03-01,en,US,"iPhone4,1",9.1,2019-03-01 17:04:46,2019-03-01,5.0,NaN,2019-03-01 19:19:52,2019-03-01,story_1,0.0,earnGemsCounter,gems,-22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11569002,22573,2019-03-05,en,GB,"iPad2,5",8.3,2019-04-07 00:07:09,2019-04-07,16.0,NaN,2019-03-11 00:15:35,2019-03-11,story_2,0.0,earnGemsCounter,gems,-22.0
11569003,22574,2019-03-02,en,US,"iPhone8,1",9.1,2019-03-02 18:22:19,2019-03-02,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11569004,22574,2019-03-02,en,US,"iPhone8,1",9.1,2019-03-02 18:22:38,2019-03-02,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11569005,22575,2019-03-03,en,US,"iPhone7,2",9.2.1,2019-03-03 19:14:35,2019-03-03,1.0,NaN,2019-03-03 19:18:00,2019-03-03,story_1,0.0,earnGemsCounter,gems,-22.0
